<a href="https://colab.research.google.com/github/thisisRMak/2025-tech16-LLM/blob/main/Lectures/Class4_TECH16_LLM_Lecture4_prepared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install sentence-transformers
!pip install langchain pypdf langchain-openai #tiktoken chromadb

# Standard imports


In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

# RAG

In [ ]:
!pip install llama-index --upgrade

In [ ]:
!pip install pypdf

In [ ]:
!wget https://www.goldmansachs.com/pdfs/insights/podcasts/episodes/ai-tom-acemoglu-covello/transcript.pdf

--2025-02-26 13:37:01--  https://www.goldmansachs.com/pdfs/insights/podcasts/episodes/ai-tom-acemoglu-covello/transcript.pdf
Resolving www.goldmansachs.com (www.goldmansachs.com)... 23.61.161.144
Connecting to www.goldmansachs.com (www.goldmansachs.com)|23.61.161.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘transcript.pdf.1’

transcript.pdf.1        [ <=>                ] 133.94K  --.-KB/s    in 0.03s   

2025-02-26 13:37:01 (4.01 MB/s) - ‘transcript.pdf.1’ saved [137153]



In [ ]:
from google.colab import userdata
open_ai_key = userdata.get('open_ai_key')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

In [ ]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = SimpleDirectoryReader("./").load_data("*.pdf")

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
index = VectorStoreIndex.from_documents(documents)

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
response = query_engine.query("Is AI overhyped?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2024 outlook.
print(response)

Loading files: 100%|██████████| 2/2 [00:01<00:00,  1.71file/s]


AI may be considered overhyped due to concerns about its potential impact and cost-effectiveness.


In [ ]:
response = query_engine.query("What did I just ask you?")
print(response)

You just asked about the potential capabilities and limitations of artificial intelligence, particularly in comparison to human cognition and the role of humans in driving the development and application of AI technologies.


# Adding chat functionality

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="openai", tool_choice="query_engine_tool", verbose=False)

In [ ]:
response = chat_engine.chat("Hi")
print(response)

Hello! How can I assist you today?


In [ ]:
response = chat_engine.chat(
    "Is AI overhyped?"
)
print(response)

AI may be considered overhyped based on the skepticism expressed by experts in the field. There are concerns about the actual impact of AI on work tasks, productivity, and GDP growth, with some experts estimating that the technology may not deliver as significantly as initially projected. Additionally, the substantial cost of developing and implementing AI without clear applications identified raises questions about the return on investment and the practicality of the technology.


In [ ]:
response = chat_engine.chat(
    "What did I just ask you?"
)
print(response)

You asked, "Is AI overhyped?"


In [ ]:
response = chat_engine.chat(
    "Who says what?"
)
print(response)

The response provided was based on the analysis and insights generated by the query engine tool in response to the question "Is AI overhyped?"


### Vector DB

In [ ]:
%pip install llama-index-vector-stores-weaviate

In [ ]:
!pip install llama-index-vector-stores-chroma

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("tech16")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./").load_data("*.pdf")
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()
response = query_engine.query("is AI overhyped?")
print(response)

Loading files: 100%|██████████| 2/2 [00:00<00:00,  2.02file/s]


## Reranker

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n = 2,
    model = "BAAI/bge-reranker-base"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k = 6,
    vector_store_query_mode="default",
    node_postprocessors = [rerank],
)
response = query_engine.query("Is AI overhyped?")
print(response)

AI is considered by some to be overhyped due to concerns about the significant costs involved in its development and implementation, especially considering the lack of clear applications for it at present. Some experts are skeptical about the potential impact of AI, estimating that only a fraction of tasks will be cost-effective to automate in the near future, leading to more modest projections for its influence on productivity and GDP growth.


# Query over structured data

In [ ]:
%pip install llama-index-program-openai

In [ ]:
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index-experimental

In [ ]:
from llama_index.experimental.query_engine import PandasQueryEngine
import pandas as pd
from vega_datasets import data
from llama_index.core import PromptTemplate

# initialize empty df
df = data.movies()
df.head()

In [ ]:
query_engine = PandasQueryEngine(df=df, verbose=True)
prompts = query_engine.get_prompts()
print(prompts["pandas_prompt"].template)

In [ ]:
new_prompt = PromptTemplate(
    """\
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
The dataframe contains data about movies.
This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}
Query: {query_str}

Return the answer from the dataframe with a natural language explanation of the answer.
Expression: """
)

query_engine.update_prompts({"pandas_prompt": new_prompt})

In [ ]:
response = query_engine.query(
    "Which movies made the most income and who was the director?",
)
print(str(response))

# Routing

In [ ]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex
from llama_index.core import Settings

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = SimpleDirectoryReader("./").load_data("transcript.pdf")

# initialize settings (set chunk size)
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()


In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to the data source",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to the data source",
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)


In [ ]:
response = query_engine.query("Is AI overhyped?")
print(response)

In [ ]:
response = query_engine.query("Summarize the document in 4 bullet points")
print(response)